In [1]:
!nvidia-smi

# imports
import math
import wave
import struct
import os
import urllib.request
import tarfile
import random
from audiolm_pytorch import SoundStream, SoundStreamTrainer, HubertWithKmeans, SemanticTransformer, SemanticTransformerTrainer, HubertWithKmeans, CoarseTransformer, CoarseTransformerWrapper, CoarseTransformerTrainer, FineTransformer, FineTransformerWrapper, FineTransformerTrainer, AudioLM
from torch import nn
import torch
import torchaudio
from random import shuffle
from fs.osfs import OSFS
from fs.mountfs import MountFS
from fs import open_fs

sample_rate = 16000

# define all dataset paths, checkpoints, etc
dataset_folder = "placeholder_dataset"
soundstream_ckpt = "results/soundstream.8.pt" # this can change depending on number of steps
hubert_ckpt = 'hubert/hubert_base_ls960.pt'
hubert_quantizer = f'hubert/hubert_base_ls960_L9_km500.bin' # listed in row "HuBERT Base (~95M params)", column Quantizer

Sat Jul 29 18:45:40 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.86.01              Driver Version: 536.67       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4070 Ti     On  | 00000000:2D:00.0  On |                  N/A |
|  0%   43C    P5              19W / 285W |   1295MiB / 12282MiB |     10%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

/opt/conda/envs/audiolm/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-07-29 18:45:42 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2023-07-29 18:45:42 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX


In [2]:
import math
from IPython.display import Audio
import matplotlib.pyplot as plt

def plot_waveform(waveform, sample_rate, title="Waveform", xlim=None):
    waveform = waveform.numpy()

    num_channels, num_frames = waveform.shape
    time_axis = torch.arange(0, num_frames) / sample_rate

    figure, axes = plt.subplots(num_channels, 1)
    if num_channels == 1:
        axes = [axes]
    for c in range(num_channels):
        axes[c].plot(time_axis, waveform[c], linewidth=1)
        axes[c].grid(True)
        if num_channels > 1:
            axes[c].set_ylabel(f"Channel {c+1}")
        if xlim:
            axes[c].set_xlim(xlim)
    figure.suptitle(title)
    plt.show(block=False)
    
def plot_specgram(waveform, sample_rate, title="Spectrogram", xlim=None):
    waveform = waveform.numpy()

    num_channels, _ = waveform.shape

    figure, axes = plt.subplots(num_channels, 1)
    if num_channels == 1:
        axes = [axes]
    for c in range(num_channels):
        axes[c].specgram(waveform[c], Fs=sample_rate)
        if num_channels > 1:
            axes[c].set_ylabel(f"Channel {c+1}")
        if xlim:
            axes[c].set_xlim(xlim)
    figure.suptitle(title)
    plt.show(block=False)

In [3]:
# Setup filesystem
dataset_fs = MountFS()

#dataset_fs.mount('dns4-read', open_fs('s3://music-clip-dataset/dns/clean-fullband/read_speech/'))
dataset_fs.mount('dns4-read', open_fs('s3://music-clip-dataset/dns/clean-fullband/russian/M-AILABS_Speech_Dataset/ru_RU_47hrs_48k/female/hajdurova/chetvero_nischih/wavs/'))
[dataset_fs.mount(f'Jamendo-{_:02x}', open_fs(f's3://music-clip-dataset/clips/train/{_:02x}/')) for _ in range(2)]
[dataset_fs.mount(f'Psychostream-{_:02x}', open_fs(f's3://music-clip-dataset/psychostream/train/{_:02x}/')) for _ in range(2)]
#dataset_fs.add_fs('dns4-read', open_fs('s3://music-clip-dataset/dns/clean-fullband/read_speech/'))

# Get all music files and shuffle them
music_files = []
for path in dataset_fs.walk.files(filter=['*.mp3']):
    music_files.append(path)
    #print(path)
random.shuffle(music_files)
print(music_files[:50])

['/Psychostream-01/44536bb07723119c07d04f9732220c01.train.5.mp3', '/Jamendo-01/090c64401a83942711bb3da6f94a2301.train.3.mp3', '/Psychostream-01/3d2d9aa22842c848c7c56648d3c46501.train.0.mp3', '/Psychostream-01/dd5c2e5f150031711f8024783fe7e501.train.0.mp3', '/Jamendo-01/d3c405a0e58c6cbcf4d5c7b92c22e401.train.15.mp3', '/Psychostream-00/6619abba734e8b4d43363ab493d7b300.train.11.mp3', '/Psychostream-01/e1bc8ff1fc6b10e3d6ee42dbae2a1601.train.6.mp3', '/Psychostream-01/c9becafffb917ea5297feb4e68c1d001.train.6.mp3', '/Psychostream-00/51aec0af93441fa6cf9e56f143646a00.train.4.mp3', '/Psychostream-00/7c68aec79306b285ff3d14d7e337d900.train.0.mp3', '/Psychostream-01/6549304a3911a6b35ec43f578a8f7d01.train.1.mp3', '/Jamendo-00/8ac545fc1ed40fd620851fd457869c00.train.5.mp3', '/Psychostream-01/628fe96d3033cd2ac1fd47b792f94201.train.8.mp3', '/Psychostream-00/8aa60049d98222ccf7e9eae7108b7000.train.0.mp3', '/Jamendo-00/d74846b8190cad0912ffd05a02e75000.train.8.mp3', '/Psychostream-00/f090002461c0d9dd3f482b6f

In [4]:
from email.mime import audio
from io import BytesIO
import os
import random
import itertools
from einops import rearrange
import fs.info
from torchaudio.functional import resample
from torch import Tensor
import torch.nn.functional as F
from audiolm_pytorch.soundstream import cast_tuple
from audiolm_pytorch.utils import curtail_to_multiple

class SeabassIterableDataset(torch.utils.data.IterableDataset):
    def __init__(self, start, end, files: list[str], fs: MountFS, effects, clip_len:float = 2.0, tick_char='.', sample_rate=16000):
        super(SeabassIterableDataset).__init__()
        self.start = start
        self.end = end
        self.effects = effects
        self.files = files
        self.fs = fs
        self.clip_len = clip_len
        self.tick_char = tick_char
        self.sample_rate = sample_rate

    def getSample(self, index):
        global sample_rate
        
        filename = self.files[index]
        #data = BytesIO(self.fs.readbytes(filename))
        with self.fs.open(filename, 'rb') as f:
            reader = torchaudio.io.StreamReader(src = f)
            reader.add_basic_audio_stream(
                frames_per_chunk=self.sample_rate//10,
                stream_index=0,
                sample_rate=self.sample_rate,
            )

            audio_tensor = None
            for i, waveform in enumerate(reader.stream()):
                if audio_tensor is None:
                    audio_tensor = waveform[0]
                else:
                    audio_tensor = torch.cat((audio_tensor, waveform[0]), 0)
                
                streamed_len = (audio_tensor.shape[0] / self.sample_rate) 
                if streamed_len >= self.clip_len:
                    break # we have enough data
               
            audio_tensor = audio_tensor.reshape(1, -1) 
            if audio_tensor.shape[0] > 1:
                # the audio has more than 1 channel, convert to mono
                audio_tensor = torch.mean(audio_tensor, dim=0).unsqueeze(0)

            #num_outputs = 1
            #audio_tensor = cast_tuple(audio_tensor, num_outputs)
            #data_tuple = tuple((resample(d, sample_hz, target_sample_hz) if exists(target_sample_hz) else d) for d, target_sample_hz in zip(data, self.target_sample_hz))

            output = []
            audio_length = audio_tensor.size(1)

            ## pad or curtail
            #max_length = 99999999999999
            #if audio_length > max_length:
            #    max_start = audio_length - max_length
            #    start = torch.randint(0, max_start, (1, ))
            #    audio_tensor = audio_tensor[:, start:start + max_length]
            #else:
            #    audio_tensor = F.pad(audio_tensor, (0, max_length - audio_length), 'constant')

            audio_tensor = rearrange(audio_tensor, '1 ... -> ...')
            output = audio_tensor.float()

            print(self.tick_char, end='', flush=True)
            return output

    
    def __iter__(self):
        worker_info = torch.utils.data.get_worker_info()
        if worker_info is None:  # single-process data loading, return the full iterator
            iter_start = self.start
            iter_end = self.end
        else:  # in a worker process
            # split workload
            per_worker = int(math.ceil((self.end - self.start) / float(worker_info.num_workers)))
            worker_id = worker_info.id
            iter_start = self.start + worker_id * per_worker
            iter_end = min(iter_start + per_worker, self.end)

        for i in range(iter_start, iter_end):
            sample = self.getSample(i)
            yield sample

# Setup dataloader
effects = [
    ["norm", "1"],
]

needed_clip_len = float(320 * 40)/sample_rate

train_subset = music_files[:-1000]
val_subset = music_files[-1000:]

dst = SeabassIterableDataset(start=0, end=len(train_subset), files=train_subset, 
                            fs=dataset_fs, effects=effects, clip_len=needed_clip_len)
dsv = SeabassIterableDataset(start=0, end=len(val_subset), files=val_subset, 
                            fs=dataset_fs, effects=effects, clip_len=2.0, tick_char='*')


In [5]:
# Test the sampler
#dl_train.sampler
#sampler = ds_train.__iter__()
#sample = sampler.__next__()
#print(sample)
#print(waveform1.shape, sr1)
#plot_waveform(waveform1, sr1, title="Train", xlim=(-0.1, 3.2))
#plot_specgram(waveform1, sr1, title="Train", xlim=(0, 3.04))
#Audio(waveform1, rate=sr1)
#
#_, waveform2, sr2 = ds_val.__iter__()
#print(waveform2.shape, sr2)
#plot_waveform(waveform2, sr2, title="Val", xlim=(-0.1, 3.2))
#plot_specgram(waveform2, sr2, title="Val", xlim=(0, 3.04))
#Audio(waveform2, rate=sr2)


### SoundStream

In [6]:
soundstream = SoundStream(
    codebook_size = 1024,
    rq_num_quantizers = 8,
)

trainer = SoundStreamTrainer(
    soundstream,
    #folder = dataset_folder,
    #train_dataloader=dl_train,
    #val_dataloader=dl_val,
    train_dataset=dst,
    val_dataset=dsv,
    #lr = 0.001,
    batch_size = 6,
    grad_accum_every = 10,         # effective batch size of 32
    data_max_length = 320 * 32,
    save_results_every = 10,
    save_model_every = 100,
    num_train_steps = 2000,
).cuda()
# NOTE: I changed num_train_steps to 9 (aka 8 + 1) from 10000 to make things go faster for demo purposes
# adjusting save_*_every variables for the same reason

trainer.train()

..............................................................................................................................0: soundstream total loss: 67.118, soundstream recon loss: 0.084 | discr (scale 1) loss: 2.000 | discr (scale 0.5) loss: 2.001 | discr (scale 0.25) loss: 2.000
******0: saving to results
0: saving model to results
........................................................................................................................1: soundstream total loss: 69.197, soundstream recon loss: 0.071 | discr (scale 1) loss: 1.993 | discr (scale 0.5) loss: 1.996 | discr (scale 0.25) loss: 1.996
........................................................................................................................2: soundstream total loss: 67.665, soundstream recon loss: 0.048 | discr (scale 1) loss: 1.993 | discr (scale 0.5) loss: 1.996 | discr (scale 0.25) loss: 1.997
....................................................................................................

KeyboardInterrupt: 

## Inference

In [ ]:
# Everything together


In [ ]:
output_path = "out.wav"
sample_rate = 44100
torchaudio.save(output_path, generated_wav.cpu(), sample_rate)